# **Implementation of a Super-Resolution Convolutional Neural Network (SRNN)**

### References
*   Image Super-Resolution using Deep Convolutional Neural Networks (paper)
*   https://vlearningit.wordpress.com/google-colab-tutorial-on-srcnn (tutorial)

### Steps
*   Import datasets
*   Create architecture
*   Utility Functions
*   Train model
*   Evaluate model

## Import Datasets

In [0]:
!git clone https://github.com/luizmanke/image-super-resolution.git
!mkdir -p data/input
!cp image-super-resolution/Datasets/Yang\ et\ al/*.bmp data/input

## Create Architecture

In [0]:
!pip install scipy==1.2.1

In [0]:
# Import system libraries
import cv2
import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
import keras.backend as K
from keras import optimizers
from keras.layers import Activation, Conv2D, Dense, Input
from keras.models import load_model, Model, Sequential
from scipy.misc import imread

In [0]:
# Build SRCNN model
img_shape = (32, 32, 1)
input_ = Input(shape=(img_shape))
C1 = Conv2D(64, (9, 9), padding='SAME', name='CONV1')(input_)
A1 = Activation('relu', name='ACT1')(C1)
C2 = Conv2D(32, (1, 1), padding='SAME', name='CONV2')(A1)
A2 = Activation('relu', name='ACT2')(C2)
C3 = Conv2D(1, (5, 5), padding='SAME', name='CONV3')(A2)
A3 = Activation('relu', name='ACT3')(C3)
model = Model(input_, A3)
opt = optimizers.Adam(lr=0.0003)
model.compile(optimizer=opt, loss='mean_squared_error')
model.summary()

## Utilitiy Functions

In [0]:
# Generate High Resolution from interpolation technique to pass in SRCNN model
def modcrop(image, scale=2):
  if len(image.shape) == 3:
    h, w, _ = image.shape
    h = h - np.mod(h, scale)
    w = w - np.mod(w, scale)
    image = image[0:h, 0:w, :]
  else:
    h, w = image.shape
    h = h - np.mod(h, scale)
    w = w - np.mod(w, scale)
    image = image[0:h, 0:w]
  return image

In [0]:
# Generate Low Resolution
def create_LR(image, scale):
  label_ = modcrop(image, scale)
  label_ = label_ / 255.  # Must be normalized
  input_ = scipy.ndimage.interpolation.zoom(label_, (1./scale), prefilter=False)
  input_ = scipy.ndimage.interpolation.zoom(input_, (scale/1.), prefilter=False)
  return input_

## Train Model

In [0]:
# Call data path in Google Colab and read all files
path = 'data/input/'
files = glob.glob(path + '*.bmp')

In [0]:
# Split data into training and test files
train_pct = 0.8
n_train = int(len(files) * train_pct)
train_files = files[:n_train]
val_files = files[n_train:]

In [0]:
# Initial parameters setting
img_size = 32
stride = 16
scale = 4
x_train = []
y_train = []
x_val = []
y_val = []

# Extract patch image for training
for file in train_files:
  tmp_y = scipy.misc.imread(file, flatten=True, mode='YCbCr').astype(np.float)
  tmp_x = create_LR(tmp_y, scale)
  h, w = tmp_y.shape
  for x in range(0, h-img_size+1, stride):
    for y in range(0, w-img_size+1, stride):
      sub_input = tmp_x[x:x+img_size, y:y+img_size].reshape(img_size, img_size, 1)
      sub_label = tmp_y[x:x+img_size, y:y+img_size].reshape(img_size, img_size, 1)
      x_train.append(sub_input)
      y_train.append(sub_label)
x_train = np.array(x_train)
y_train = np.array(y_train)

In [0]:
# Extract patch image for validation
for file in val_files:
  tmp_y = scipy.misc.imread(file, flatten=True, mode='YCbCr').astype(np.float)
  tmp_x = create_LR(tmp_y,scale)
  h,w = tmp_y.shape
  for x in range(0, h-img_size+1, stride):
    for y in range(0, w-img_size+1, stride):
      sub_input = tmp_x[x:x+img_size, y:y+img_size].reshape(img_size, img_size, 1) # [32 x 32]
      sub_label = tmp_y[x:x+img_size, y:y+img_size].reshape(img_size, img_size, 1) # [32 x 32]
      x_val.append(sub_input)
      y_val.append(sub_label)
x_val = np.array(x_val)
y_val = np.array(y_val)

In [0]:
# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=1, validation_data=(x_val, y_val))

## Evaluate Model

In [0]:
# Predict
y_pred = model.predict(x_train)

In [0]:
# MSE
def compute_mse(img1, img2):
  mse = np.mean((img1 - img2) ** 2)
  return mse

mse = compute_mse(x_train[0], y_train[0])

In [0]:
# PSNR
def compute_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
      return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

psnr = compute_psnr(x_train[0], y_train[0])

In [0]:
# SSIM
from skimage.measure import compare_ssim as ssim